## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.20,-175.20,79.38,83,100,11.97,moderate rain
1,1,Faanui,PF,-16.48,-151.75,81.57,70,11,15.01,light rain
2,2,Carlsbad,US,33.16,-117.35,58.95,89,100,5.75,mist
3,3,Lazaro Cardenas,MX,17.96,-102.20,71.15,72,5,4.16,clear sky
4,4,Dingle,PH,11.00,122.67,78.28,83,63,6.60,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.20,-175.20,79.38,83,100,11.97,moderate rain
1,1,Faanui,PF,-16.48,-151.75,81.57,70,11,15.01,light rain
4,4,Dingle,PH,11.00,122.67,78.28,83,63,6.60,broken clouds
5,5,Puerto Ayora,EC,-0.74,-90.35,76.55,100,100,7.58,light rain
17,17,Isangel,VU,-19.55,169.27,80.71,86,91,7.07,overcast clouds
26,26,Hithadhoo,MV,-0.60,73.08,83.46,69,7,5.41,clear sky
27,27,Kloulklubed,PW,7.04,134.26,75.70,89,100,13.51,heavy intensity rain
38,38,Arraial Do Cabo,BR,-22.97,-42.03,84.16,78,75,10.36,broken clouds
43,43,Hervey Bay,AU,-25.30,152.85,75.02,76,61,8.99,broken clouds
44,44,Avarua,CK,-21.21,-159.78,75.25,94,0,2.30,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.20,-175.20,79.38,83,100,11.97,moderate rain
1,1,Faanui,PF,-16.48,-151.75,81.57,70,11,15.01,light rain
4,4,Dingle,PH,11.00,122.67,78.28,83,63,6.60,broken clouds
5,5,Puerto Ayora,EC,-0.74,-90.35,76.55,100,100,7.58,light rain
17,17,Isangel,VU,-19.55,169.27,80.71,86,91,7.07,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
698,698,Koster,ZA,-25.86,26.90,78.22,21,2,5.23,clear sky
701,701,Aden,YE,12.78,45.04,81.30,63,0,8.48,clear sky
703,703,Manavalakurichi,IN,8.13,77.30,83.23,77,21,6.26,few clouds
704,704,Richards Bay,ZA,-28.78,32.04,81.16,65,13,8.28,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,79.38,moderate rain,-21.20,-175.20,
1,Faanui,PF,81.57,light rain,-16.48,-151.75,
4,Dingle,PH,78.28,broken clouds,11.00,122.67,
5,Puerto Ayora,EC,76.55,light rain,-0.74,-90.35,
17,Isangel,VU,80.71,overcast clouds,-19.55,169.27,
26,Hithadhoo,MV,83.46,clear sky,-0.60,73.08,
27,Kloulklubed,PW,75.70,heavy intensity rain,7.04,134.26,
38,Arraial Do Cabo,BR,84.16,broken clouds,-22.97,-42.03,
43,Hervey Bay,AU,75.02,broken clouds,-25.30,152.85,
44,Avarua,CK,75.25,clear sky,-21.21,-159.78,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,79.38,moderate rain,-21.20,-175.20,Keleti Beach Resort
1,Faanui,PF,81.57,light rain,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
4,Dingle,PH,78.28,broken clouds,11.00,122.67,SEGAYA PROPERTY
5,Puerto Ayora,EC,76.55,light rain,-0.74,-90.35,Finch Bay Galapagos Hotel
17,Isangel,VU,80.71,overcast clouds,-19.55,169.27,Tanna Lodge


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))